# **Récupération et formatage des données**
---


## Introduction<a class="anchor" id="partie1"></a>

Ce notebook contient les codes nécessaires à la récupération et au formatage des données. Les données ont été récupéré sur le site de [l'OCDE](https://data-explorer.oecd.org/) et sur celui [d'Eurostat](https://ec.europa.eu/eurostat/fr/data/database). 

Nous récupérons les données en csv et les transformons en un unique csv contenant pour chacun des pays les valeurs trimestrielles (de Q1 en 1995 à Q3 de 2024 <span style="color:red ; font-size:20px;"> faire attention les working hours ne vont qu'au Q3 de 2024 </span>) de PIB, d'heure de travail, des taux d'intérêts à court et long terme et de l'indice à la consommation des prix. Pour l'instant il nous manque aussi les anticipations des taux et les rendements des marchés financiers. Le csv final de ce notebook servira de base de données par la suite. 

Les pays retenus sont ceux pour lesquels on a réussi à collecter toutes les données relatives aux variables d'intérêt: Autriche (Austria) Belgique (Belgium) Bulgarie (Bulgaria) Tchéquie (Czechia) Danemark (Denmark) Estonie (Estonia) Finlande (Finland) France (France) Allemagne (Germany) Grèce (Greece) Hongrie (Hungary) Islande (Iceland) Irlande (Ireland) Italie (Italy) Lettonie (Latvia) Lituanie (Lithuania) Luxembourg (Luxembourg) Pays-Bas (Netherlands) Norvège (Norway) Pologne (Poland) Portugal (Portugal) Roumanie (Romania) Slovaquie (Slovakia) Slovénie (Slovenia) Espagne (Spain) Suède (Sweden) Suisse (Switzerland) Royaume-Uni (United Kingdom).

<span style="color:red ; font-size:20px;"> Compléter avec les variables manquantes et mettre une liste des pays (si on ne la met pas ailleurs) </span>

## Sommaire

* [Introduction](#partie1)
* [Récupération des données](#partie2)
    * [Données de PIB](#partie21)
    * [Données d'heures de travail](#partie22)
    * [Données des taux d'intérêts](#partie23)


## Récupération des données <a class="anchor" id="partie2"></a>

In [71]:
!pip install -r requirements.txt

In [72]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

Nous créons la liste qui contient les pays cités précédemment, à noter que la Slovaquie apparaît sous le nom Slovakia pour Eurostat et Slovak Republic pour l'OCDE, elle est donc sous deux noms dans notre liste de pays à sélectionner. 

In [73]:
#Liste comportant les pays choisis
countries_selected=[
    "Austria", "Belgium", "Bulgaria", "Czechia", "Denmark", "Estonia", 
    "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Netherlands", 
    "Norway", "Poland", "Portugal", "Romania", "Slovakia", "Slovak Republic",
    "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom"
]

### Données de PIB <a class="anchor" id="partie21"></a>

Nous extrayons les données relatives au PIB depuis Eurostat


In [74]:
# Charge les données depuis le csv 
df_GDP = pd.read_csv("Données_extraites/GDP_trimestriel_eurostat.csv", encoding='utf-8')

# Sélectionne les colonnes nécessaires avec les pays sélectionnés
df_GDP_selected = df_GDP[df_GDP['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Change le format avec les périodes en index et les noms des pays en noms de colonnes
df_GDP_pivot = df_GDP_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transforme le nom des colonnes en PIB_Nom_du_pays
df_GDP_pivot.columns = [f'PIB_{col}' for col in df_GDP_pivot.columns]
df_GDP_pivot.columns = [col.replace(" ", "_") for col in df_GDP_pivot.columns]

### Données de d'heure de travail <a class="anchor" id="partie22"></a>

Nous extrayons les données relatives aux heures de travail depuis Eurostat. Nous avons trois base de données pour les heures de travail. Une base annuelle de 1995 à 1997, et deux bases trimestrielles allant du Q1 de 1998 au Q4 de 2007 et du Q1 de 2008 au Q3 de 2024.

Nous voulons traiter les données dans l'ordre chronologique. Nous devons donc pour la première période récupérer les données annuelles et les transformer en trimestrielles, nous choisissons d'interpoler les données manquantes de manière linéaire en donnant la valeur annuelle au Q1. Pour cela nous avons besoin de traiter les données de 1998 à 2007 avant afin de pouvoir faire l'interpolation entre le Q1 de 1997 et celui de 1998.

Nous récupérons donc d'abord toutes les données


In [75]:
# Charge les données de 1995 à 1997 
df_working_hours_95_97 = pd.read_csv("Données_extraites/Working_hours_eurostat_1995-1997.csv", encoding='utf-8')

# Sélectionne les colonnes nécessaires avec les pays sélectionnés
df_working_hours_95_97_selected = df_working_hours_95_97[df_working_hours_95_97['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Change le format avec les périodes en index et les noms des pays en noms de colonnes
df_working_hours_95_97_pivot = df_working_hours_95_97_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transforme le nom des colonnes en WH_Nom_du_pays
df_working_hours_95_97_pivot.columns = [f'WH_{col}' for col in df_working_hours_95_97_pivot.columns]
df_working_hours_95_97_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_95_97_pivot.columns]

# Même processus pour celles de 1998 à 2007 et celles de 2008 à 2024
df_working_hours_98_07 = pd.read_csv("Données_extraites/Working_hours_eurostat_1998-2007.csv", encoding='utf-8')
df_working_hours_98_07_selected = df_working_hours_98_07[df_working_hours_98_07['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]
df_working_hours_98_07_pivot = df_working_hours_98_07_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')
df_working_hours_98_07_pivot.columns = [f'WH_{col}' for col in df_working_hours_98_07_pivot.columns]
df_working_hours_98_07_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_98_07_pivot.columns]

df_working_hours_08_24 = pd.read_csv("Données_extraites/Working_hours_eurostat_2008-2024.csv", encoding='utf-8')
df_working_hours_08_24_selected = df_working_hours_08_24[df_working_hours_08_24['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]
df_working_hours_08_24_pivot = df_working_hours_08_24_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')
df_working_hours_08_24_pivot.columns = [f'WH_{col}' for col in df_working_hours_08_24_pivot.columns]
df_working_hours_08_24_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_08_24_pivot.columns]



Et maintenant nous configurons l'interpolation. Pour clarifier le point précédent, l'interpolation est linéaire, donc va tracer une droite entre deux périodes avec une valeur présente et inférer une valeur aux périodes pour laquelle la valeur est manquante et qui se trouvent entre nos deux périodes. Comme nous faisons le choix de donner la valeur annuelle au Q1 il faut ajouter une valeur de 1998 pour pouvoir inférer les valeurs du Q2, Q3 et Q4 de 1997, ce que nous faisons donc en ajoutant le Q1 de 1998 à notre table

In [76]:
# Récupération du Q1 de 1998 et suppresion de la table pour éviter les doublons lors de la concaténation
Q1_1998 =  df_working_hours_98_07_pivot.loc[['1998-Q1']]
df_working_hours_98_07_pivot = df_working_hours_98_07_pivot.drop(index='1998-Q1', errors='ignore')

# On ajoute les trimestres vierges aux données de 1995 à 1998 pour ensuite les compléter par interpolation
new_index = []
values = []
for year in df_working_hours_95_97_pivot.index:
    # Ajouter Q1 pour chaque année (avec la valeur annuelle)
    new_index.append(f"{year}-Q1")
    values.append(df_working_hours_95_97_pivot.loc[year].values)
    
    # Ajouter Q2, Q3 et Q4 (lignes vides pour l'instant)
    for quarter in ['Q2', 'Q3', 'Q4']:
        new_index.append(f"{year}-{quarter}")
        values.append([None] * len(df_working_hours_95_97_pivot.columns))

# Création du dataframe avec les lignes vides
df_working_hours_95_97_pivot = pd.DataFrame(values, index=new_index, columns=df_working_hours_95_97_pivot.columns)
df_working_hours_95_97_pivot.index.name = 'TIME_PERIOD'

df_working_hours_95_97_pivot = pd.concat([df_working_hours_95_97_pivot, Q1_1998])

# Interpolation
df_working_hours_95_97_pivot = df_working_hours_95_97_pivot.interpolate(method='linear')

Maintenant nous procédons à la concaténation des données pour avoir un unique dataframe contenant les heures de travail trimestrielles de 1995 à 2024.

In [77]:
# Concaténation
df_working_hours = pd.concat(
    [df_working_hours_95_97_pivot, df_working_hours_98_07_pivot, df_working_hours_08_24_pivot], 
    axis=0
)

### Données des taux d'intérêts <a class="anchor" id="partie23"></a>

Pour les taux d'intérêts nous récupérons des taux de long-terme, à savoir des obligations d'Etat à 10 ans, et des taux de court-terme. Les taux à court terme sont généralement soit le taux interbancaire à trois mois lié aux prêts accordés et acceptés par les banques pour tout excès ou manque de liquidité sur plusieurs mois, soit le taux associé aux bons du Trésor, aux certificats de dépôt ou à des instruments comparables, chacun d'une durée de trois mois. Pour les pays de la zone euro, le « European Interbank Offered Rate » à trois mois est utilisé à partir de la date à laquelle le pays a rejoint l'euro.  

Nous commençons par récupérer les taux de long-terme puis ceux de court-terme

In [78]:
# Charger les données
df_LT_IR = pd.read_csv('Données_extraites/Long_term_IR_OCDE.csv', encoding='utf-8')

# Sélectionne les colonnes nécessaires avec les pays sélectionnés
df_LT_IR_selected = df_LT_IR[df_LT_IR['Reference area'].isin(countries_selected)][['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Change le format avec les périodes en index et les noms des pays en noms de colonnes
df_LT_IR_pivot = df_LT_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE')

# Renommer les colonnes pour inclure le préfixe LT_IR_ (=Long Term Interest Rate)
df_LT_IR_pivot.columns = [f'LT_IR_{col}' for col in df_LT_IR_pivot.columns]
df_LT_IR_pivot.columns = [col.replace(" ", "_") for col in df_LT_IR_pivot.columns]

# Même processus pour les taux de long-terme
df_ST_IR=pd.read_csv('Données_extraites/Short_term_IR_OCDE.csv', encoding='utf-8')
df_ST_IR_selected = df_ST_IR[df_ST_IR['Reference area'].isin(countries_selected)][['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]
df_ST_IR_pivot = df_ST_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE')
df_ST_IR_pivot.columns = [f'ST_IR_{col}' for col in df_ST_IR_pivot.columns]
df_ST_IR_pivot.columns = [col.replace(" ", "_") for col in df_ST_IR_pivot.columns]